In [12]:
import numpy as np
import os
import sys
from tensorflow.keras import models
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import requests
import datetime
from bs4 import BeautifulSoup
from PIL import Image
import pyodbc
import cv2

In [13]:
url = 'https://pportal.hpa.gov.tw/Web/Notice.aspx'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.maximize_window()
time.sleep(1)
driver.find_element('xpath','//*[@id="ctl00_pnlLogin"]/button').click()
time.sleep(1)
print("網址設定成功")

driver.save_screenshot('screen.jpg')
element =driver.find_element('xpath','//*[@id="imgCheckCode"]')
left = element.location['x']
top = element.location['y']
right = left + element.size['width']
bottom = top + element.size['height']
img  = Image.open('screen.jpg')
img = img.crop((left, top, right, bottom))
img = img.convert("RGB")
img.save('numbercheck.jpg', 'jpeg')
img = img.convert("L")  # 將圖片轉成灰階
pix_img = img.load()
w, h = img.size
threshold = 205 #像素閥值
for y in range(h):    #for迴圈將每個像素點進行判斷
    for x in range(w):
        if pix_img[x, y] < threshold:
            pix_img[x, y] = 0
        else:
            pix_img[x, y] = 255
data = img.getdata()
w, h = img.size
black_point = 0
for x in range(1, w - 1):
    for y in range(1, h - 1):
        mid_pixel = data[w * y + x]
        if mid_pixel < 50:
            top_pixel = data[w * (y - 1) + x]
            left_pixel = data[w * y + (x - 1)]
            down_pixel = data[w * (y + 1) + x]
            right_pixel = data[w * y + (x + 1)]
            if top_pixel < 10:
                black_point += 1
            if left_pixel < 10:
                black_point += 1
            if down_pixel < 10:
                black_point += 1
            if right_pixel < 10:
                black_point += 1
            if black_point < 1:
                img.putpixel((x, y), 255)
            black_point = 0
img.save(f'C:/Users/dl/Desktop/yc_data/babyCrawler/CaptchaJpg/Train/captcha.jpg')
print("已儲存:captcha.jpg")
#driver.quit()

C:\Users\dl\AppData\Local\Temp\ipykernel_21248\2229208928.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


網址設定成功
已儲存:captcha.jpg


In [14]:
img_rows = None
img_cols = None
digits_in_img = 4
model = None
np.set_printoptions(suppress=True, linewidth=150, precision=9, formatter={'float': '{: 0.9f}'.format})

In [15]:
def split_digits_in_img(img_array):
    x_list = list()
    for i in range(digits_in_img):
        step = img_cols // digits_in_img
        x_list.append(img_array[:, i * step:(i + 1) * step] / 255)
    return x_list

In [16]:
if os.path.isfile('cnn_model.h5'):
    model = models.load_model('cnn_model.h5')
else:
    print('No trained model found.')
    exit(-1)

In [17]:
img = load_img(f'C:/Users/dl/Desktop/yc_data/babyCrawler/CaptchaJpg/Train/captcha.jpg', color_mode='grayscale')
img_array = img_to_array(img)
img_rows, img_cols, _ = img_array.shape
x_list = split_digits_in_img(img_array)

In [18]:
varification_code = list()
for i in range(digits_in_img):
    confidences = model.predict(np.array([x_list[i]]), verbose=0)
    result_class = np.argmax(confidences, axis=1)
    varification_code.append(result_class[0])
    print('Digit {0}: Confidence=> {1}    Predict=> {2}'.format(i + 1, np.squeeze(confidences), np.squeeze(result_class)))
print('Predicted verification code:', varification_code)


Digit 1: Confidence=> [ 0.999990702  0.000000407  0.000000020  0.000004126  0.000000016  0.000000143  0.000000720  0.000000775  0.000000004  0.000003093]    Predict=> 0
Digit 2: Confidence=> [ 0.000000450  0.000000002  0.000000000  0.000000000  0.000000000  0.000000001  0.000000000  0.999999523  0.000000000  0.000000000]    Predict=> 7
Digit 3: Confidence=> [ 0.000000000  0.000000000  0.000000000  0.000000278  0.000000000  0.999996066  0.000003663  0.000000000  0.000000004  0.000000000]    Predict=> 5
Digit 4: Confidence=> [ 0.000000363  0.000000002  0.999992609  0.000000000  0.000000000  0.000000000  0.000000000  0.000006323  0.000000089  0.000000637]    Predict=> 2
Predicted verification code: [0, 7, 5, 2]


In [19]:
CAPTCHA = ''.join(str(code) for code in varification_code[:4])

In [20]:
CAPTCHA

'0752'

In [21]:
img

In [22]:
driver.find_element('id','ctl00_txtAccount').send_keys("ayi2000719")
driver.find_element('id','ctl00_txtPassword').send_keys("Kimlu###0719")
driver.find_element('id','ctl00_txtVerification').send_keys(CAPTCHA)
time.sleep(0.5)
driver.find_element('xpath','//*[@id="ctl00_lbtnLogin"]').click()